# Built-in Functions

| ANSI SQL Standard | POstgreSQL Extensions |
--------- | -------
sum  | trunc
max | cbrt
min | pi
cast | random
count | to_num
floor | to_char
ceiling | now


## Business Use Cases

+ Encaptsulate business logic
+ Enforce semantic relationships
+ Validate and transoform data
+ control access to sensitive date
+ Encourage code reuse
+ Don't Repeat YOurself(DRY)




+ Setting up a working environment
+ Creating your first function
+ Data types
+ Polymorphic functions
+ Function overloading
+ Stored procedures
+ Error handling

In [1]:
select CURRENT_TIMESTAMP

(1 row(s) affected)

Total execution time: 00:00:00.002

current_timestamp
2024-09-07 23:42:02.844127+08


In [2]:
select * from customers limit 1

(1 row(s) affected)

Total execution time: 00:00:00.002

customerid,firstname,lastname,address1,address2,city,state,zip,country,region,email,phone,creditcardtype,creditcard,creditcardexpiration,username,password,age,income,gender
1,RNPOJG,FQIVBKWIZC,1287944454 Dell Way,,XCXNWIS,ND,42572,US,1,FQIVBKWIZC@dell.com,1287944454,5,1890860486775636,2012/06,user1,password,49,80000,M


In [4]:
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10;

(10 row(s) affected)

Total execution time: 00:00:00.002

firstname,lastname,orderid,netamount
RKLQYY,HPUIUYBWZX,10677,5.08
GWPSHI,QWRLBUPCHY,2337,39.06
FCHDNP,FQZNEVVHNA,9077,323.30
HOMUAY,BCTJTZIEZQ,6239,341.44
UZCYBH,NKKNSRDQGC,1187,285.39
YADYNS,VBVFOAENRH,3710,350.87
ZGYPBI,NKKQMREHPQ,379,227.45
ZGYPBI,NKKQMREHPQ,9447,83.31
PFIWAF,MQGNVHRTNH,3075,33.63
PQJVZM,PCCNFVKLZE,5019,256.30


In [5]:
create or replace view cust_orders 
as 
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10;

CREATE VIEW

Total execution time: 00:00:00.003

In [18]:
select * from cust_orders;

: relation "cust_orders" does not exist

Total execution time: 00:00:00

In [7]:
create function cust_orders()
    returns table (firstname varchar, lastname varchar, orderid int, netamount numeric)
    as $$
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [8]:
select * from cust_orders();

(10 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname,orderid,netamount
RKLQYY,HPUIUYBWZX,10677,5.08
GWPSHI,QWRLBUPCHY,2337,39.06
FCHDNP,FQZNEVVHNA,9077,323.30
HOMUAY,BCTJTZIEZQ,6239,341.44
UZCYBH,NKKNSRDQGC,1187,285.39
YADYNS,VBVFOAENRH,3710,350.87
ZGYPBI,NKKQMREHPQ,379,227.45
ZGYPBI,NKKQMREHPQ,9447,83.31
PFIWAF,MQGNVHRTNH,3075,33.63
PQJVZM,PCCNFVKLZE,5019,256.30


In [11]:
create or replace function cust_orders(customerid int)
    returns table (firstname varchar, lastname varchar, orderid int, netamount numeric)
    as $$
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid 
    where c.customerid = cust_orders.customerid
    limit 10
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [17]:
select * from cust_orders(42);

(1 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname,orderid,netamount
IGAITZ,ADIDJMAYUZ,11465,202.29


In [19]:
drop view if exists cust_orders;
drop function if exists cust_orders();
drop function if exists cust_orders(int);

NOTICE: view "cust_orders" does not exist, skipping

DROP VIEW

DROP FUNCTION

DROP FUNCTION

Total execution time: 00:00:00.002

## Input Parameters and Return Values

### Multiple input parameters


In [21]:
create or replace function my_pow(x double precision, y double precision)
returns double precision
as $$ 
select power(x, y)
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [23]:
select my_pow(1,2)

(1 row(s) affected)

Total execution time: 00:00:00.002

my_pow
1


In [25]:
select my_pow(PI(), log(42))

(1 row(s) affected)

Total execution time: 00:00:00.001

my_pow
6.412068866243689


### Default values

In [27]:
create or replace function my_default(x int = 42) 
RETURNS INT
as $$
    SELECT x;
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [28]:
select my_default()

(1 row(s) affected)

Total execution time: 00:00:00.003

my_default
42


### Using arrays for multiple input values

In [32]:
create or replace function array_sum(int_array int[])
RETURNs int 
as $$
    select sum(el) 
    from unnest(int_array) as arr(el);
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [34]:
select array_sum(array[1,2,3])

(1 row(s) affected)

Total execution time: 00:00:00.001

array_sum
6


## Arrays and multiple return values


In [37]:
create or replace function array_sum_avg(int_array int[])
RETURNS table (array_sum bigint, array_avg numeric)
as $$
select sum(el), avg(el)::numeric(5,2)
from unnest(int_array) as arr(el);
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [39]:
select array_sum_avg(array[1,2,3]) as "Record type";



(1 row(s) affected)

Total execution time: 00:00:00.001

Record type
"(6,2.00)"


In [41]:
select * from array_sum_avg(array[1,2,3])

(1 row(s) affected)

Total execution time: 00:00:00.001

array_sum,array_avg
6,2.00


## Output arguments


In [3]:
create or replace function get_cust_name (
    in id int,
    out firstname varchar,
    out lastname varchar
) as $$
    select c.firstname, c.lastname from customers C
    where c.customerid = id;
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [6]:
SELECT * from get_cust_name(42);

(1 row(s) affected)

Total execution time: 00:00:00.007

firstname,lastname
IGAITZ,ADIDJMAYUZ


## Returns a table revisited

In [7]:
create or replace function get_cust_names(id1 int, id2 int) 
returns table (firstname varchar, lastname varchar)
as $$
select c.firstname, c.lastname from customers C
where c.customerid between id1 and id2;
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [10]:
select * from get_cust_names(1, 5);

(5 row(s) affected)

Total execution time: 00:00:00.003

firstname,lastname
RNPOJG,FQIVBKWIZC
RKLQYY,HPUIUYBWZX
GWPSHI,QWRLBUPCHY
ONCPHI,TZIJOMZQJJ
CIDXWX,GNYGKZXSCR


In [14]:
-- drop function if exists get_cust_names(id1 int, id2 int)

create or replace function get_cust_names2(
    inout id1 int, id2 int,
    out firstname varchar,
    out lastname varchar
) as $$
select c.customerid, c.firstname, c.lastname from customers c
where c.customerid between id1 and id2
-- order by c.customerid desc
$$ language sql

CREATE FUNCTION

Total execution time: 00:00:00.001

In [16]:
select 
id1 as customerid,
firstname, 
lastname
 from get_cust_names2(1,5);

(1 row(s) affected)

Total execution time: 00:00:00.001

customerid,firstname,lastname
1,RNPOJG,FQIVBKWIZC


In [17]:

select 
id1 as customerid,
firstname, 
lastname
 from get_cust_names2(-1,-25);

(1 row(s) affected)

Total execution time: 00:00:00.001

customerid,firstname,lastname
NULL,NULL,NULL


In [19]:
select 
firstname, 
lastname
 from get_cust_names(-1,-25);

(0 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname


In [ ]:
drop function if EXISTS get_cust_name(id int, out firstname varchar, out lastname varchar);
drop function if EXISTS get_cust_names(id1 int, id1 int, out firstname varchar, out lastname varchar);
drop function if EXISTS get_cust_names(input id1 int, id1 int, out firstname varchar, out lastname varchar);
drop function if EXISTS get_cust_names(id1 int, id1 int);

## Calling Functions

In [20]:
create or replace function call_me(x int, y int, sw boolean = True) returns INT
as $$
select x + y where sw 
union ALL
select x - y where not sw;
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.003

In [21]:
select call_me(42, -42 ) as "Positional arguments",
call_me(x := 42, y := -42) as "Named arguments",
call_me(42, -42, False) as "Positional arguments with switch",
call_me(42, -42, sw := False) as "Mixed positional and named";

(1 row(s) affected)

Total execution time: 00:00:00.002

Positional arguments,Named arguments,Positional arguments with switch,Mixed positional and named
0,0,84,84


In [23]:
select call_me(x := 42, y := -42, False) as "Named Followed by positional";

: positional argument cannot follow named argument

Total execution time: 00:00:00

In [2]:
create or replace function call_me2(x int, y int, sw boolean = false) returns INT
as $$
select x + y where sw
union ALL
select x - y where not sw
$$ language sql;

CREATE FUNCTION

执行时间总计: 00:00:00

In [3]:
select call_me2(42, -42 ) as "Positional arguments",
call_me2(x := 42, y := -42) as "Named arguments",
call_me2(42, -42, False) as "Positional arguments with switch",
call_me2(42, -42, sw := False) as "Mixed positional and named";

(1 row(s) affected)

执行时间总计: 00:00:00.024

Positional arguments,Named arguments,Positional arguments with switch,Mixed positional and named
84,84,84,84


In [4]:
SELECT
    c.column_name,
    pgd.description
FROM
    pg_catalog.pg_statio_all_tables as st
    INNER JOIN pg_catalog.pg_description pgd ON (pgd.objoid = st.relid)
    INNER JOIN information_schema.columns c ON (pgd.objsubid = c.ordinal_position
        AND c.table_schema = st.schemaname AND c.table_name = st.relname)

(11 row(s) affected)

执行时间总计: 00:00:00.010

column_name,description
jobagentid,Agent that currently executes this job.
jstkind,"Kind of jobstep: s=sql, b=batch"
jstonerror,"What to do if step returns an error: f=fail the job, s=mark step as succeeded and continue, i=mark as fail but ignore it and proceed"
jlgstatus,"Status of job: r=running, s=successfully finished, f=failed, i=no steps to execute, d=aborted"
jslstatus,"Status of job step: r=running, s=successfully finished, f=failed stopping job, i=ignored failure, d=aborted"
jslresult,Return code of job step
employee_id,员工ID
first_name,名字
last_name,姓氏
email,邮箱


## Creating Functions in PL/pgSQL

Using Pl/pgSQL

+ Process blocks of computation on the server
+ Includes control structures for iteration and conditional processing
+ Supports typed variables for complex calculations
+ Inherits all user-defined types, functions, and operators
+ Procecdural language for creating functions and triggers


## Structure of a PL/pgSQL program



### Returning a table using the SQL language

In [2]:
create or replace FUNCTION get_cust_names(id1 int, id2 int)
returns table (firstname varchar, lastname varchar) AS
$$
select c.firstname, c.lastname from customers as C
where c.customerid BETWEEN id1 and id2
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

### Returing a table using teh PL/pgSQL language


In [3]:
create or replace function get_cust_names(id1 int, id2 int)
returns table(firstname varchar, lastname varchar)
as $$
BEGIN
    return query
    select c.firstname, c.lastname from customers as c
    where c.customerid BETWEEN id1 and id2;
end
$$ language plpgsql;

CREATE FUNCTION

Total execution time: 00:00:00.005

In [4]:
SELECT * from get_cust_names(1,5);

(5 row(s) affected)

Total execution time: 00:00:00.004

firstname,lastname
RNPOJG,FQIVBKWIZC
RKLQYY,HPUIUYBWZX
GWPSHI,QWRLBUPCHY
ONCPHI,TZIJOMZQJJ
CIDXWX,GNYGKZXSCR


### PL/pgSQL program structure



```
[<<label>>]
[ declare
    declarations ]
begin
    statements
end [label];
```

In [5]:
do $$ begin null;end; $$

DO

Total execution time: 00:00:00.001

In [7]:
do language plpgsql $$
<<get_ans>>
DECLARE
    the_answer int := 42;
begin
    raise notice 'The answer is %.', get_ans.the_answer;
end get_ans;
$$

NOTICE: The answer is 42.

DO

Total execution time: 00:00:00

### Declaring and using variables


#### Delcaring variables

In [10]:
do LANGUAGE plpgsql $$
-- name [constant ] type [ collate collation_name ] [not null] [ (default | := | =) expression ];

DECLARE
    myint INTEGER not null = 0;
    the_answer NUMERIC(2) := 42;
    phi CONSTANT double precision DEFAULT ( 1 + |/ 5) / 2;
    cust_row customers%rowtype;
    cust_firstname customers.firstname%type;
    myrow record;
BEGIN
    myint := -1; --myint = -1
    select * from customers into cust_row limit 1;
    select cust_row.firstname into cust_firstname limit 1;
    select 42::int as the_answer into myrow;
    raise notice 'Last name: %; First name: %; How young? %', cust_row.lastname, cust_firstname, myrow.the_answer;
    end;
    $$

NOTICE: Last name: FQIVBKWIZC; First name: RNPOJG; How young? 42

DO

Total execution time: 00:00:00.003

### Executing Queries

#### Executing queries in Pl/pgSQL

why does this fail?


In [12]:
create or replace function foo() returns table(the_answer int)
LANGUAGE plpgsql as $$
begin 
select 42 as "The answer";
end;
$$;

SELECT * from foo();

CREATE FUNCTION

: query has no destination for result data

Total execution time: 00:00:00.001

In [13]:
create or replace function foo() returns table(the_answer int)
LANGUAGE plpgsql as $$
begin 
return query
select 42 as "The answer";
end;
$$;

SELECT * from foo();

CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.002

the_answer
42


In [15]:

create or replace function foo() returns table(the_answer int)
LANGUAGE plpgsql as $$
begin 
perform 42 as "The answer";
end;
$$;
select foo();


CREATE FUNCTION

(0 row(s) affected)

Total execution time: 00:00:00.002

foo


#### Returning that value


In [17]:
create or replace function bar(out a int)
LANGUAGE plpgsql as $$
begin 
select 42 as "The answer" into bar.a;
end;
$$;
select bar();

CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.003

bar
42


#### INTO clause

select select_statements into [strict] target from ...;

insert ... returing expressions into [strict] target; -- strict means must return 1 row, otherwise exception will raise

iupdate ... returing expressions into [strict] target;

delete ... returing expressions into [strict] target;


In [18]:
do $$
DECLARE
    the_answer int;
    cust_row customers%rowtype;
    firstname VARCHAR;
    lastname VARCHAR;
    new_cat int;
    old_catname varchar;
BEGIN
    select 42 into the_answer;
    select * into cust_row from customers c where c.customerid = 42;
    select c.firstname, c.lastname into firstname, lastname
    from customers c where c.customerid = 42;
    INSERT into categories (categoryname) values ('MyCategory') returning category into new_cat;
    delete from categories where category = new_cat returning categoryname into old_catname;
    raise notice 'id: %, name: %', new_cat, old_catname;
END
$$

NOTICE: id: 17, name: MyCategory

DO

Total execution time: 00:00:00.006

In [21]:
create or replace function foo2() RETURNS table(firstname varchar)
LANGUAGE plpgsql as $$
BEGIN
return QUERY
    update customers c
    set firstname = c.firstname
    where c.customerid = 42
    returning c.firstname;
    end
    $$;

select foo2();

do $$ begin perform foo2(); end; $$;

CREATE FUNCTION

(1 row(s) affected)

DO

Total execution time: 00:00:00.004

foo2
IGAITZ


### Control Structures


#### Returing values Scalar results

In [26]:
create or replace function foo3() returns INT
LANGUAGE plpgsql as $$
BEGIN
    return 42;
    END
    $$;

select foo3();

CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.002

foo3
42


In [28]:
create or replace function foo3(out i int)  -- areturns int
LANGUAGE plpgsql as $$
BEGIN
    i = 42
    return; --i
END
$$;

select foo3();


CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.002

foo3
42


#### Returning a result set: RETURN QUERY and RETURN NEXT

In [29]:
drop FUNCTION if EXISTS foo(out i int);
create or REPLACE function foo() returns setof INT
LANGUAGE plpgsql as $$
BEGIN

RETURN next 42;
return next 43;
END;
$$;

select * from foo();

DROP FUNCTION

CREATE FUNCTION

(2 row(s) affected)

Total execution time: 00:00:00.004

foo
42
43


#### Returning a set of a table type

In [31]:
create or REPLACE FUNCTION get_cat()
returns setof categories
-- returns table (category int, categoryname varchar)
language plpgsql as $$
DECLARE 
    cat categories%rowtype;
BEGIN
    cat.category := 1;cat.categoryname := 'One';
    return next cat;
    cat.category := 2; cat.categoryname := 'Tow';
    return next cat;
end;
$$;
select * from get_cat();

CREATE FUNCTION

(2 row(s) affected)

Total execution time: 00:00:00.003

category,categoryname
1,One
2,Tow


### Conditional execution: IF statement

```sql
IF ... THEN END  IF
IF ... THEN ELSE END  IF
IF ... THEN ELSIF THEN ELSE END  IF
IF ... THEN END  IF

```

In [32]:
do $$ begin
    if true THEN
        raise notice 'True';
    ELSE
        raise notice 'False';
    end if;
end;
$$;]

NOTICE: True

DO

Total execution time: 00:00:00

In [33]:
do $$ begin
if 1 < 0 THEN
    raise notice 'One is less then zero?';
elsif 1 > 0 THEN
    raise notice 'One is greater then zero!';
else 
    raise notice 'How did I get here?';
end if;
end;
$$;

NOTICE: One is greater then zero!

DO

Total execution time: 00:00:00

#### Conditional execution Simple CASE statement


```sql
case search-expression
    when expression [, expresion [...] ] then
        statements
    [ when expression [, expresion [...] ] then 
        statements
        ...
    ]
    [ ELSE 
        statements
    ]

END CASE;
```

In [34]:
do $$ begin
case 42
    when 41, 43 THEN
        raise notice 'Odd.';
        when 40, 42 THEN
        raise notice 'Even.';
        end case;
        end;
        $$;

NOTICE: Even.

DO

Total execution time: 00:00:00.001

#### Conditional execution: Searched CASE statement

```sql
case 
    when boolean-expression then
    statements
    [ when boolean-expression then 
        statements
        ...
    ]
    [
        else
        statements

    ]
    end case;

```

In [35]:
do $$ begin
    case 
        when 42 in (41, 43) THEN raise notice 'Odd.';
        when 42 in (40, 42) THEN raise notice 'Even.';
    end CASE;
end;
$$;


NOTICE: Even.

DO

Total execution time: 00:00:00

### Iterations

#### Simple loops


In [38]:
DO
$$

    DECLARE
        i int := 1;
    BEGIN
        <<loop_label>>
        LOOP
            RAISE NOTICE 'in a loop iteration : %', i;
            i = i + 1;
            IF i > 3 THEN
                EXIT loop_label;
            END IF;
        END LOOP loop_label;
        RAISE NOTICE 'Finally out!';
    END
$$;


NOTICE: in a loop iteration : 1 \n
NOTICE: in a loop iteration : 2 \n
NOTICE: in a loop iteration : 3 \n
NOTICE: Finally out!

DO

Total execution time: 00:00:00

In [ ]:
DO
$$

    DECLARE
        i int := 1;
    BEGIN
        LOOP
            RAISE NOTICE 'In a loop, iteration: %', i;
            i = i + 1;
            EXIT WHEN i > 3;
            CONTINUE WHEN i <= 8;
        END LOOP;
        RAISE NOTICE 'Finally out!';
    END;
$$


### Iterations


#### While loops

In [39]:
DO
$$
    DECLARE
        i int := 1;
    BEGIN
        WHILE i <> 3
            LOOP
                RAISE NOTICE 'In a loop iteration %', i;
                i = i + 1;
            END LOOP;
        RAISE NOTICE 'Finally out!';
    END;
$$


NOTICE: In a loop iteration 1
NOTICE: In a loop iteration 2
NOTICE: Finally out!

DO

Total execution time: 00:00:00.001

### for loops, integer iterator

```sql
[<<labe.>>]
    for name in [ reverse ] expression ... expression [by expression ] loop
        statments
    end loop [ label ];

```

In [40]:
DO
$$
    <<outside>>
        DECLARE
        i int = 42;
    BEGIN
        <<inside>>
        FOR i IN 1..3
            LOOP
                RAISE NOTICE 'In a loop iteration %, outer "i" %', inside.i, outside.i;
            END LOOP;
        RAISE NOTICE 'Finally out';
    END;
$$


NOTICE: In a loop iteration 1, outer "i" 42
NOTICE: In a loop iteration 2, outer "i" 42
NOTICE: In a loop iteration 3, outer "i" 42
NOTICE: Finally out

DO

Total execution time: 00:00:00.001

#### Optional arguments for integer iterators


```sql
for i in reverse 1..3 loop
    -- i set to 5,4,3,2,1
end loop

for i in 1..5 loop by 2
    --i set to 1 , 3, 5
    end loop

```

#### for loops: looping over query results



In [41]:
DO
$$
    DECLARE
        i customers.customerid%type;
    BEGIN
        FOR i IN
            SELECT c.customerid
            FROM customers c
            WHERE c.customerid <= 3
            LOOP
                RAISE NOTICE 'In a loop, iteration %', i;
            END LOOP;
        RAISE NOTICE 'Finally out';
    END;
$$


NOTICE: In a loop, iteration 1
NOTICE: In a loop, iteration 2
NOTICE: In a loop, iteration 3
NOTICE: Finally out

DO

Total execution time: 00:00:00.001

#### For loops: Looping through arrays

In [42]:
DO
$$
    DECLARE
        i int;
    BEGIN
        FOREACH i IN ARRAY ARRAY [11,22,33]
            LOOP
                RAISE NOTICE 'In a loop, value %', i;
            END LOOP;
        raise NOTICE  'finally OUt!';
    END;
$$


NOTICE: In a loop, value 11
NOTICE: In a loop, value 22
NOTICE: In a loop, value 33
NOTICE: finally OUt!

DO

Total execution time: 00:00:00

### Using Cursors


#### Iteration using cursors

##### Bound cursors

In [43]:
DO
$$

    DECLARE
        cur CURSOR FOR SELECT *
                       FROM customers;
        rc int = 3;
    BEGIN
        FOR rec IN cur
            LOOP
                IF rc > 0 THEN
                    RAISE INFO '(%, %)', rec.customerid, rec.firstname;
                END IF;
                rc = rc - 1;
                IF rc = 0 THEN
                    EXIT;
                END IF;
            END LOOP;
    END;
$$


INFO: (1, RNPOJG)
INFO: (2, RKLQYY)
INFO: (3, GWPSHI)

DO

Total execution time: 00:00:00.001

##### Unbounded cursor

In [44]:
DO
$$

    DECLARE
        cur refcursor;
        fn  customers.firstname%type;
    BEGIN
        OPEN cur FOR SELECT customers.firstname FROM customers;
        FOR rc IN 1..3
            LOOP
                FETCH NEXT FROM cur INTO fn;
                RAISE INFO '(%,%)', rc, fn;
                rc = rc + 1;
            END LOOP;
        CLOSE cur;
    END;
$$;


INFO: (1,RNPOJG)
INFO: (2,RKLQYY)
INFO: (3,GWPSHI)

DO

Total execution time: 00:00:00

#### Cursor with parameter

In [45]:
CREATE OR REPLACE FUNCTION get_cur(prefix varchar)
    RETURNS refcursor
    LANGUAGE plpgsql AS
$$
DECLARE
    cur CURSOR (p varchar) FOR
        SELECT *
        FROM customers
        WHERE firstname LIKE p || '%';
BEGIN
    OPEN cur(prefix);
    RETURN cur;
END;
$$;

BEGIN;
SELECT get_cur('AZ');
FETCH 5 FROM cur;
COMMIT;


CREATE FUNCTION

BEGIN

(1 row(s) affected)

(5 row(s) affected)

COMMIT

Total execution time: 00:00:00.005

get_cur
cur


customerid,firstname,lastname,address1,address2,city,state,zip,country,region,email,phone,creditcardtype,creditcard,creditcardexpiration,username,password,age,income,gender
829,AZPVTW,UMTJYQQCKY,5533373880 Dell Way,,UGZCMII,MN,27018,US,1,UMTJYQQCKY@dell.com,5533373880,2,5252180055715143,2011/11,user829,password,46,60000,M
1322,AZRCMO,GBRDIKHGZH,3140327323 Dell Way,,HKTOUMX,OH,57042,US,1,GBRDIKHGZH@dell.com,3140327323,4,4135095870587975,2010/05,user1322,password,46,100000,M
2025,AZMAIQ,HFBBETZROB,4168882685 Dell Way,,MDABECE,LA,29423,US,1,HFBBETZROB@dell.com,4168882685,2,5058186975307762,2012/02,user2025,password,24,60000,M
2356,AZWSBL,ZYXCHTOWRG,6155980013 Dell Way,,CNUBGNZ,KY,27719,US,1,ZYXCHTOWRG@dell.com,6155980013,4,5126263682264834,2012/01,user2356,password,69,20000,F
5016,AZNUCI,MTUYHSTFJN,1021093327 Dell Way,,HXDCKNP,NJ,73053,US,1,MTUYHSTFJN@dell.com,1021093327,2,6551118154544383,2009/10,user5016,password,45,40000,M


Row by agonizing row (RBAR) - Jeff Moden

### Dynamic queries


In [46]:
DO
$$
    DECLARE
        i int;
    BEGIN
        -- strict ensure only one row is returned, otherwise error raised
        --EXECUTE 'select 42 where false' INTO STRICT i;
--         EXECUTE 'select 42 union select 43' INTO STRICT i;
        EXECUTE 'select 42' INTO STRICT i;
        RAISE NOTICE 'i = %', i;
    END;
$$


NOTICE: i = 42

DO

Total execution time: 00:00:00

#### Dynamic table names and qoute_ident funciton

In [55]:
DO
$$
    DECLARE
        tablename  name;
        tablenames name[] = ARRAY ['customers', 'orders'];
        rc         bigint;
    BEGIN
        tablename = tablenames[1 + RANDOM()];
--         SELECT COUNT(*) FROM tablename INTO rc;
        execute 'select count(*) from' || quote_ident(tablename) into rc;
        RAISE NOTICE '% has % rows', tablename, rc;
    END;
$$


NOTICE: orders has 1 rows

DO

Total execution time: 00:00:00.001

#### Quoting Iterals with quote_literal and quote_nullable

In [56]:
DO
$$
    DECLARE
        firstname  varchar;
        firstnames varchar[] = ARRAY ['A', 'B'];
        tbl_name   name      = 'customers';
        col_name   name      = 'firstname';
        rc         bigint;
    BEGIN
        firstname = firstnames[1 + RANDOM()];
        EXECUTE
            'select count(*) from '
                || QUOTE_IDENT(tbl_name)
                || ' where '
                || QUOTE_IDENT(col_name)
                || ' like '
                || QUOTE_LITERAL(firstname || '%') INTO rc;
        RAISE NOTICE '% firstnames start with %', rc, firstname;

    END;
$$;


NOTICE: 747 firstnames start with A

DO

Total execution time: 00:00:00.004

In [57]:
SELECT lastname,
       'quoted: ' || QUOTE_LITERAL(lastname),
       'nullable:' || QUOTE_NULLABLE(lastname)
FROM (SELECT '0''Neil' AS lastname UNION SELECT NULL) s;


(2 row(s) affected)

Total execution time: 00:00:00.001

lastname,Column2,Column3
0'Neil,quoted: '0''Neil',nullable:'0''Neil'
NULL,NULL,nullable:NULL


### SQL injection and USING clause

In [58]:
DROP TABLE IF EXISTS foo;
CREATE TABLE foo
(
    a int
);
DO
$$
    DECLARE
        baddata varchar = '0; drop table if exists foo;';
    BEGIN
        EXECUTE 'select count(*) from orders where orderid = '
            || baddata;
    END;
$$;

SELECT *
FROM information_schema.tables
WHERE table_name = 'foo';


DO
$$
    DECLARE
        firstname  varchar;
        firstnames varchar[] = ARRAY['A', 'B'];
        tbl_name   name      = 'customers';
        col_name   name      = 'firstname';
        rc         bigint;
    BEGIN
        firstname = firstnames[1 + RANDOM()];
        EXECUTE
            'select count(*) from '
                || QUOTE_IDENT(tbl_name)
                || ' where '
                || QUOTE_IDENT(col_name)
                || ' like $1'
            INTO rc
            USING firstname || '%';
        RAISE NOTICE '% firstnames start with "%"', rc, firstname;
    END;
$$;



NOTICE: table "foo" does not exist, skipping

DROP TABLE

CREATE TABLE

DO

(0 row(s) affected)

NOTICE: 801 firstnames start with "B"

DO

Total execution time: 00:00:00.012

table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action


In [59]:
DO
$$
    DECLARE
        firstname  varchar;
        firstnames varchar[] = ARRAY['A', 'B'];
        tbl_name   name      = 'customers';
        col_name   name      = 'firstname';
        rc         bigint;
    BEGIN
        firstname = firstnames[1 + RANDOM()];
        EXECUTE
            'select count(*) from '
                || QUOTE_IDENT(tbl_name)
                || ' where '
                || QUOTE_IDENT(col_name)
                || ' like $1'
            INTO rc
            USING firstname || '%';
        RAISE NOTICE '% firstnames start with "%"', rc, firstname;
    END;
$$;


NOTICE: 747 firstnames start with "A"

DO

Total execution time: 00:00:00.003

In [64]:
DO
$$
    DECLARE
        firstname  varchar;
        firstnames varchar[] = ARRAY ['A', 'B'];
        tbl_name   name      = 'customers';
        col_name   name      = 'firstname';
        rc         bigint;
    BEGIN
        firstname = firstnames[1 + RANDOM()];
        EXECUTE
            FORMAT(
                    'select count(*) from %I where %I like $1',
                    tbl_name,
                    col_name
            ) INTO rc USING firstname || '%';
        RAISE NOTICE '% firstnames start with "%"', rc, firstname;
    END;
$$;


NOTICE: 747 firstnames start with "A"

DO

Total execution time: 00:00:00.005

#### Other uses for dynamic queries


```sql
return query execute command-string [using expression [, ...]];
for target in execute text_expression [using expression [, ...]] loop ...
open unbound_cursorvar [ [ no] scroll] for execute query_string

```

## Creating Polymorphic Functions to Control Repetition

+ Polymorphism support
+ Ad hot
+ Subtyping
+ Parametric(generic)
+ Polymorphism support
+ Variadic
+ Default parameter and ambiguity



## Ad hoc polymorphic functions


#### Adding two numbers


In [65]:
CREATE OR REPLACE FUNCTION adhoc(x int, y int) RETURNS int
    LANGUAGE sql AS
'SELECT x + y;';

CREATE OR REPLACE FUNCTION adhoc(x float, y float) RETURNS float
    LANGUAGE sql AS
'SELECT x + y;';

CREATE OR REPLACE FUNCTION phi() RETURNS float
    -- golden ratio
    LANGUAGE sql AS
'SELECT (1::float + SQRT(5::float)) / 2::float;';

SELECT adhoc(7, 4), adhoc(PI(), phi());

SELECT adhoc('A'::char, 'B'::char);


CREATE FUNCTION

CREATE FUNCTION

CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.002

adhoc,adhoc
11,4.759626642339688


: function adhoc(character, character) does not exist

#### Selecting from different tables based on parameter types


In [68]:
CREATE OR REPLACE FUNCTION getcol(age customers.age%type) RETURNS int
    LANGUAGE sql AS
$$
SELECT customerid
FROM customers c
WHERE c.age = getcol.age;
$$;

CREATE OR REPLACE FUNCTION getcol(id orders.orderid%type) RETURNS numeric
    LANGUAGE sql AS
$$
SELECT totalamount
FROM orders
WHERE orderid = id;
$$;

SELECT getcol(age) AS customer_id
FROM customers limit 1;

SELECT getcol(age::int) AS "customer_id (is it?)"
FROM customers limit 1;
SELECT getcol(orderid::int2)
FROM orders limit 1;


NOTICE: type reference customers.age%TYPE converted to smallint

CREATE FUNCTION

NOTICE: type reference orders.orderid%TYPE converted to integer

CREATE FUNCTION

(1 row(s) affected)

...

(1 row(s) affected)

Total execution time: 00:00:00.008

customer_id
1


customer_id (is it?)
24.26


getcol
NULL


### Subtype polymorphic functions


In [69]:
CREATE TABLE expedited_orders
(
    expedited_date date NOT NULL
) INHERITS (orders);

INSERT INTO expedited_orders (orderdate, customerid, netamount, tax, totalamount, expedited_date)
VALUES ('20201225', 1, 0, 0, 0, '20200601');

SELECT *
FROM expedited_orders;

CREATE OR REPLACE FUNCTION has_tax(o_row orders)
    RETURNS table
            (
                tbl varchar,
                tax boolean
            )
    LANGUAGE sql
AS
$$
SELECT 'orders', o_row.tax = 0;
$$;

CREATE OR REPLACE FUNCTION has_tax(o_row expedited_orders)
    RETURNS table
            (
                tbl varchar,
                tax boolean
            )
    LANGUAGE sql
AS
$$
SELECT 'expedited_orders', o_row.tax = 0;
$$;

SELECT has_tax(o.*) from orders o where orderid = 1;
SELECT has_tax(e.*) from expedited_orders e;


: relation "expedited_orders" already exists

Total execution time: 00:00:00

### Parametric Parameter Functions


Pseudo types


| Name | Description| Note |
|--------- | ------------- |-----
any | any input data type | Not for SQL or Pl/pgSQL
anyarray | any array type
anyelement | any data datatype 
anyenum | any enum type
anynonarray | any non array type
void | returns no value


#### Generic substraction

In [70]:
CREATE OR REPLACE FUNCTION f(x anyelement, y anyelement)
    RETURNS anyelement
    LANGUAGE sql
AS
'SELECT x - y';

SELECT pg_typeof(f(42, 42)), f(42,42);
SELECT pg_typeof(f(42.0, 42.0)), f(42.0,42.0);
SELECT pg_typeof(f(pi(), phi())), f(pi(),phi());


CREATE FUNCTION

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.004

pg_typeof,f
integer,0


pg_typeof,f
numeric,0.0


pg_typeof,f
double precision,1.5235586648398982


In [71]:
select f(true, false);


: operator does not exist: boolean - boolean

Total execution time: 00:00:00.001

#### Generic ARRAY 'addition" (concatenation)

In [72]:
CREATE OR REPLACE FUNCTION g(x anyarray, y anyarray)
    RETURNS anyarray
    LANGUAGE sql AS
'SELECT x || y';

SELECT g(ARRAY [1,2,3], array[4,5,6]);
SELECT g(ARRAY ['a', 'b'], array['c', 'd']);


CREATE FUNCTION

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.003

g
"{1,2,3,4,5,6}"


g
"[""a"", ""b"", ""c"", ""d""]"


#### Unsupported type

In [73]:
CREATE OR REPLACE FUNCTION h(x anyenum)
    RETURNS void
    LANGUAGE sql AS
'';
SELECT h(42);


CREATE FUNCTION

: function h(integer) does not exist

Total execution time: 00:00:00.001

#### anyelement with restrictions

In [74]:
CREATE OR REPLACE FUNCTION f(x anyelement, y anyelement)
    RETURNS anyelement
    LANGUAGE plpgsql AS
$$
BEGIN
    IF pg_typeof(x) IN ('integer', 'numeric', 'double precision')
        AND pg_typeof(y) IN ('integer', 'numeric', 'double precision')
    THEN
        RETURN x - y;
    ELSE
        RETURN NULL;
    END IF;
END;
$$;

SELECT f(42, 42) AS "ints", f(42.0, 42.0) AS "nums", f(42::real, 42::real) AS "reals";


CREATE FUNCTION

(1 row(s) affected)

Total execution time: 00:00:00.003

ints,nums,reals
0,0.0,NULL


### Variadic functions

#### Variadic parameter of a specific type


In [75]:
CREATE OR REPLACE FUNCTION v(VARIADIC x int[]) RETURNS setof int
    LANGUAGE sql AS
$$
SELECT *
FROM UNNEST(x);
$$;

SELECT *
FROM v(1, 2);
SELECT *
FROM v(VARIADIC x := ARRAY [1,2,3]);


CREATE FUNCTION

(2 row(s) affected)

(3 row(s) affected)

Total execution time: 00:00:00.003

v
1
2


v
1
2
3


#### Variadic parameter of a pseudo type (generic)

In [76]:
CREATE OR REPLACE FUNCTION v(VARIADIC x anyarray)
    RETURNS setof ANYELEMENT
    LANGUAGE SQL AS
$$
SELECT *
FROM UNNEST(x);
$$;


select * from v(1,2);
select * from v('a'::char, 'b'::char);

CREATE FUNCTION

(2 row(s) affected)

(2 row(s) affected)

Total execution time: 00:00:00.003

v
1
2


v
a
b


#### Variadic logger

In [77]:
CREATE OR REPLACE FUNCTION logger(VARIADIC l varchar[]) RETURNS void
    LANGUAGE plpgsql AS
$$
DECLARE
    le varchar;
BEGIN
    SELECT STRING_AGG(x, '|') FROM UNNEST(l) AS l(x) INTO le;
    RAISE NOTICE 'Log entry: %', le;
END;
$$;

SELECT logger('this', 'is','a','log', 'entry');


CREATE FUNCTION

NOTICE: Log entry: this|is|a|log|entry

(1 row(s) affected)

Total execution time: 00:00:00.002

logger
""


### Default Parameters and Ambiguity

In [78]:
CREATE OR REPLACE FUNCTION f(a int, b int) RETURNS int
    LANGUAGE sql AS
'SELECT 42';
CREATE OR REPLACE FUNCTION f(a int, b int, c int = 42) RETURNS int
    LANGUAGE sql AS
'SELECT c';

select f(42,42);
select f(42,42, 43);


CREATE FUNCTION

CREATE FUNCTION

: function f(integer, integer) is not unique

Total execution time: 00:00:00